In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Чтобы облегчить просмотр отчета большая часть кода вынесена в отдельный модуль

In [1]:
import read_file as rf
import split_data as spt
import data_preprocessing as proc

# Content-based  подход

Данный метод основывается на характеристиках, которые мы знаем о пользователях и объектах, которые хотим ему порекомендовать. В качестве характеристик пользователя может выступать любая доступная нам информация, не нарушающая этические нормы. 

Допустим, речь идет о социальной сети и мы хотим рекомендовать пользователю новых друзей. В данной задаче в качестве объектов выступают другие пользователи. В качестве характеристик пользователей можно использовать общедоступные данные из профиля пользователя, такие как: сообщества в которых состоит данный пользователь, количество его записей на своей странице, заполненность профиля и так далее. Вся эта информация поможет нашей рекомендательной системе найти близких по духу людей для данного пользователя. 

Если у нас имеется интернет-магазин, то в качестве характеристик пользователя у нас имеются данные которые пользователь указал в своем профиле, а в качестве характеристик объектов имеется любая информация, доступная о товаре.

Для пользователей-объектов у нас так же имеется рейтинг (посещения страниц других пользователей, клики по товарам, оценки фильмов). Именно на основе данного рейтинга и обучаются алгоритмы. 

Content-based подход тоже основан на таблице с оценками, которые пользователи поставили определенным фильмам. Но основная задача данного метода заключается в нахождении нужных признаков, которые дадут наилучший алгоритм предсказания будущих оценок.

Наша задача состоит в том, чтобы попробовать использовать content-basend подход на данных MovieLens.

### Считываем данные

In [3]:
df_users, df_movies, df_rates = rf.read_zipped_file('ml-1m.zip')

### Делим данные на обучение и контроль

Делить данные необходимо по времени. 
- Как это понимать? 
>> Нужно для каждого пользователя отсортировать данные по времени и поделить в некотором соотношении на обучение и контроль. 
- Зачем делать именно так? Неужели нельзя разбить глобально по времени или просто в случайном порядке?
>> Это необходимо для того, чтобы обученный алгоритм, внедренный в систему, мог приносить пользу и предсказывал верные оценки. Дело в том, что предпочтения пользователей может меняться, соответсвенно, если мы возьмем их оценки в случайном порядке, то в таком случае мы не сможем отследить тенденцию к смене жанра, например. 

In [4]:
%time df_splited = spt.get_train_test_split(df_rates, df_users.index)

CPU times: user 55.2 s, sys: 49.3 s, total: 1min 44s
Wall time: 1min 45s


Так как выборка поделилась на обучение и контроль, у нас появилось два подмножества, индексы которых не совпадают с исходным множеством. Для удобства будем хранить новые индексы отдельно для фильмов и пользователей для обучения и котроля.

In [8]:
%%time
movies_ind = spt.get_new_indexes(df_splited, 'movie_id')
users_ind = spt.get_new_indexes(df_splited, 'user_id')

CPU times: user 280 ms, sys: 6.8 ms, total: 287 ms
Wall time: 287 ms


###  Добавляем фичи

В задание нам изначально уже даны некоторые признаки, которые необходимо использовать. Опишем их:

- f<sub>u</sub><sup>1</sup><sub>,i</sub> – категориальный признак, возраст пользователя

- f<sub>u</sub><sup>2</sup><sub>,i</sub> – категориальный признак, профессия пользователя

- f<sub>u</sub><sup>3</sup><sub>,i</sub> – набор булевых признаков, по одному на каждый жанр, к которому отнесен фильм

- f<sub>u</sub><sup>4</sup><sub>,i</sub> – категориальный признак, пол пользователя

- f<sub>u</sub><sup>5</sup><sub>,i</sub> – (u<sub>g</sub> · m<sub>g</sub>)/n<sub>g</sub>, где u<sub>g</sub> – вектор средних оценок пользователя в пространстве жанров, m<sub>g</sub> – булевый вектор для фильма в пространстве жанров, n<sub>g</sub> – количество жанров, указанных для фильма

- f<sub>u</sub><sup>6</sup><sub>,i</sub> – средний рейтинг пользователя

- f<sub>u</sub><sup>7</sup><sub>,i</sub> – средний рейтинг фильма

- f<sub>u</sub><sup>8</sup><sub>,i</sub> – константный признак

Как можно заметить, несмотря на то, что мы вроде бы используем только характеристики пользователей и фильмов, мы все же не можем обойтись без исполльзования оценок пользователей и вводим признак, основывающийся на средней оценке.

Таким образом мы выявляем наиболее предпочтительные жанры

In [9]:
%%time
df_list = proc.add_features(df_splited, users_ind, movies_ind, df_users, df_movies)

------------------------------------------------------------
                Preprocessing of 1 sample
------------------------------------------------------------
------------------------------------------------------------
                Preprocessing of 2 sample
------------------------------------------------------------
CPU times: user 2min 8s, sys: 12.1 s, total: 2min 20s
Wall time: 2min 22s


Мы получили разбиение на обучение и контроль, но пока у нас все еще отсутсвуем целевая переменная. Она находится внутри матрицы объекты-признаки. Нужно это исправить.

In [11]:
y_final = df_list[0].rating, df_list[1].rating
X_final = df_list[0].drop('rating', axis=1), df_list[1].drop('rating', axis=1)

In [12]:
X_final[0].columns

Index(['user_id', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'movies_avg', 'user_avg', 'age_1', 'age_18', 'age_25', 'age_35',
       'age_45', 'age_50', 'age_56', 'gender_F', 'gender_M', 'occupation_0',
       'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12',
       'occupation_13', 'occupation_14', 'occupation_15', 'occupation_16',
       'occupation_17', 'occupation_18', 'occupation_19', 'occupation_2',
       'occupation_20', 'occupation_3', 'occupation_4', 'occupation_5',
       'occupation_6', 'occupation_7', 'occupation_8', 'occupation_9',
       'product_feature', 'const'],
      dtype='object')

### Обучаем модель

Подберем параметр для Ridge-регрессии с помощью кросс валидации:

In [13]:
def split_add_feature(n_folds, df, users_index):
    folds = spt.get_folds(df, users_index, n_folds)
    movies_folds = spt.get_new_indexes_for_folds(folds, 'movie_id')
    users_folds = spt.get_new_indexes_for_folds(folds, 'user_id')
    folds = proc.add_features(folds, users_folds, movies_folds, df_users, df_movies)
    y = [list(folds[i].rating) for i in range(n_folds)]
    
    X = []
    for i in range(n_folds):
        X.append(folds[i].drop('rating', axis=1))
    return X, y

Стоит отдельно отметить то, как происходит кросс-валидация.

В начале, все как всегда, делим выборку на n_folds фолдов. Но так как у нас довольно специфичная задача, то кросс валидация должна проходить по времени. Иначе она теряет смысл. 
- Так каким же образом происходит кросс-валидация?

>> Рассмотрим на примере кросс-валидации на 3 фолдах. 

>>Обучаемся на  <b>time[0]</b> ==> предсказываем на <b>time[1]</b>, <b>time[2]</b>

>> Обучаемся на <b>time[0]</b>, <b>time[1]</b> ==> предсказываем на <b>time[2]</b>

>> И усредняем результат. Таким образом мы всегда обучаемся на прошлых данных и предсказываем будущие.

In [14]:
def cross_validation_by_time(X, y, param, n_folds):
    one_param_loss = []
    for i in range(n_folds-1):
        df_to_learn = pd.DataFrame()
        y_to_learn = np.array([])
        for to_learn in range(i+1):
            df_to_learn = df_to_learn.append(X[to_learn])
            y_to_learn = np.append(y_to_learn, y[to_learn])
        df_to_test = pd.DataFrame()
        y_to_test = []
        for to_test in range(i+1, n_folds):
            df_to_test = df_to_test.append(X[to_test])
            y_to_test = np.append(y_to_test, y[to_test])
        regr = Ridge(alpha=param)
        regr.fit(df_to_learn, y_to_learn)
        predicted = regr.predict(df_to_test)
        cur_loss = mean_squared_error(y_true=y_to_test, y_pred=predicted)
        one_param_loss.append(cur_loss)
    return np.array(one_param_loss).mean()    

In [25]:
def print_phrase(phrase):
    print ('--' * 30)
    size = int((60-len(phrase))/2)
    print (' ' * size, phrase)
    print ('--' * 30)

Проведем кросс-валидацию на 4 фолдах, для выявления лучшего параметра

In [26]:
params = np.array([0.0001, 0.001, 0.01, 0.1])
loss = []
n_folds = 4
print_phrase('Starting by creating spliting into folds')

X, y = split_add_feature(n_folds, df_rates, df_users.index)
print_phrase('Splitting is ended')

for i in range(params.shape[0]):
    loss.append(cross_validation_by_time(X, y, params[i], n_folds))
    print_phrase('On parametr %f loss is %f' % (params[i], loss[i]))

------------------------------------------------------------
           Starting by creating spliting into folds
------------------------------------------------------------
------------------------------------------------------------
                Preprocessing of 1 sample
------------------------------------------------------------
------------------------------------------------------------
                Preprocessing of 2 sample
------------------------------------------------------------
------------------------------------------------------------
                Preprocessing of 3 sample
------------------------------------------------------------
------------------------------------------------------------
                Preprocessing of 4 sample
------------------------------------------------------------
------------------------------------------------------------
                      Splitting is ended
------------------------------------------------------------
-------

In [27]:
loss

[0.94303909607008685,
 0.94303909606432734,
 0.94303909600674141,
 0.94303909543142816]

Как видно из кросс-валидации -- изменение значения MSE при различных параметрах не очень существенное. Можно выбирать любой параметр. Хотя значение MSE на 0.1 немного меньше. Выберем этот параметр.

In [28]:
regr = Ridge(alpha=0.1)
regr.fit(X_final[0], y_final[0])
predicted = regr.predict(X_final[1])
loss = mean_squared_error(y_true=y_final[1], y_pred=predicted)

In [29]:
loss

0.93054602833777655

### Добавление признаков

#### Первая попытка

Добавим в качестве признака часть zip-code отвечающую за регион США. Исходим из предположения, что разные фильмы смотрят в разных регионах. Если это так, то MSE должен стать меньше. 

In [24]:
my = proc.add_my_features_to_folds(df_list, df_users, df_movies)

In [25]:
y_my = my[0].rating, my[1].rating
X_my = my[0].drop('rating', axis=1), my[1].drop('rating', axis=1)

In [26]:
regr = Ridge(alpha=0.1)
regr.fit(X_my[0], y_my[0])
predicted = regr.predict(X_my[1])
loss = mean_squared_error(y_true=y_my[1], y_pred=predicted)

In [27]:
loss

0.93073236866692621

При добавлении первой цифры из zip-code MSE стал чуточку больше, значит, это плохой признак

#### Вторая попытка

In [21]:
import imp
imp.reload(proc)

<module 'data_preprocessing' from '/Users/tatiana/Documents/university/6_sem/Practicum/data_preprocessing.py'>

Попробуем добавить год фильма в качестве признака

In [22]:
my = proc.add_my_features_to_folds(df_list, df_users, df_movies)

In [23]:
y_my = my[0].rating, my[1].rating
X_my = my[0].drop('rating', axis=1), my[1].drop('rating', axis=1)

In [24]:
regr = Ridge(alpha=0.1)
regr.fit(X_my[0], y_my[0])
predicted = regr.predict(X_my[1])
loss = mean_squared_error(y_true=y_my[1], y_pred=predicted)

In [25]:
loss

0.9307550888385353

Данный признак тоже не дал прирост, даже наоборот, он оказался шумовым

### Выводы

Content-based подход хорош, когда есть дополнительная информация об объектах или пользователях. При условии нахождения хороших признаков можно получить очень хороший результат. Признаки можно находить как интерпретируемые, так и не интерепретируемые (взять различные комбинации, возвести в квадрат и так далее).

Достоинства:
    - Быстро считается (при условии правильного пользования библиотеками питона)
    - Интересен с целью поиска признаков (можно очень много всего придумать и получить лучший результат)
Недостатки: 
    - Нужно потратить очень много времени, чтобы найти хорошие признаки